In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import torch
from transformers import AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import pipeline
from transformers import AutoModelForCausalLM # Zero-shot LLaMA-2-7B
from transformers import TrainingArguments
from transformers import Trainer

from google.colab import drive

from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import json


# Modify and push files
from huggingface_hub import login, logout
from huggingface_hub import HfApi

from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
catalysts = pd.read_csv('./drive/MyDrive/Colab Notebooks/final_cats.csv').drop(labels='Unnamed: 0', axis=1)
# sec = pd.read_csv('./drive/MyDrive/Colab Notebooks/sec.csv').drop(labels='Unnamed: 0', axis=1)

# Shifting catalyst Labels
Catalyst labels Map:
-1 -> 0 : BEARISH
0 -> 1: NEUTRAL
1 -> 2: BULLISH

In [ ]:
catalysts['label'] = catalysts['label'].astype(int) + 1
features = catalysts.iloc[:, :-1]
labels = catalysts.iloc[:, -1:]
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
print(f"Train Size = {X_train.shape[0]}")
print(f"Test Size = {X_test.shape[0]}")

Train Size = 3139
Test Size = 785


In [ ]:
train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
test = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)

In [ ]:
train.head()

,ticker,disease,stage,date,catalyst,label
0,AZN,Neurofibromatosis type 1 plexiform neurofibromas,Approved,4/13/20,"FDA Approval announced April 13, 2020.",2
1,GOSS,Eosinophilic asthma,Phase 2b,10/13/20,Phase 2b trial did not meet primary endpoint -...,0
2,AZN,"Risk of cardiovascular (CV) death, hospitaliza...",Approved,5/9/23,"Approved May 9, 2023.",2
3,ABBV,Upper Limb Spasticity,Approved,7/29/21,"FDA approval announced July 29, 2021.",2
4,INFI,Triple negative breast cancer (TNBC) and ovari...,Phase 1,12/9/20,Phase 1 initial data due presented at SABCS An...,1


In [ ]:
test.head()

,ticker,disease,stage,date,catalyst,label
0,RLAY,Cholangiocarcinoma (CCA),Phase 1,10/12/23,Phase 1 data from non-CCA expansion cohort pre...,2
1,GSK,Rheumatoid Arthritis,Phase 2b,10/21/18,"Phase 2b released October 21, 2018 did not mee...",0
2,ABBV,Episodic Migraine,Phase 3,4/21/23,Phase 3 trial met its primary and secondary en...,1
3,CRSP,Advanced clear cell renal cell carcinoma,Phase 1,11/10/22,Phase 1 result reported a 77% disease control ...,2
4,RHHBY,Follicular lymphoma,Phase 3,6/10/22,Phase 3 final analysis reported that PFS was s...,2


In [ ]:
train.to_csv("./drive/MyDrive/Colab Notebooks/train.csv", index=False)
test.to_csv("./drive/MyDrive/Colab Notebooks/test.csv", index=False)

In [ ]:
train.shape

(3139, 6)

In [ ]:
test.shape

(785, 6)